In [ ]:
import torch
import torch.nn as nn
from torchmetrics.classification import BinaryJaccardIndex, JaccardIndex
from torchmetrics.functional.classification import multiclass_jaccard_index
from torchmetrics.functional.segmentation import mean_iou
from torch.nn.functional import softmax, one_hot

In [ ]:
X = torch.rand(size=(1,3,3,8))
Y = (torch.rand(size=(1,3,3,8)) > 0.5) * 1

In [ ]:
print(X)
print(Y)

In [ ]:
m1 = BinaryJaccardIndex()
m2 = JaccardIndex(task='binary', average='micro')

In [ ]:
print(m1(X,Y), m2(X,Y))

In [ ]:
# s = 0
# for i in range(7):
#     t = m1(X[i], Y[i])
#     s += t
#     print(t)
# print(s, s/8)

In [ ]:
p1 = torch.mean(X, axis = (1,2))
print(p1)

In [ ]:
print(torch.mean(X[0]))

In [ ]:
# print(torch.mean(X[1]))

## Torchmetrics multiclass_jaccard_index vs mean_iou

In [ ]:
pred = softmax(torch.rand((4,21,224,224)), dim=1)
target = torch.randint(21, (4,224,224))
target_oh = torch.permute(one_hot(target, num_classes = 21), (0,3,1,2))
pred_oh = torch.permute(one_hot(torch.argmax(pred, dim=1), num_classes = 21), (0,3,1,2))

In [ ]:
mIU = mean_iou(torch.argmax(pred, dim=1), target, num_classes=21, per_class=True)
mIU1 = mean_iou(pred_oh, target_oh, num_classes=21, per_class=True)
iou = multiclass_jaccard_index(pred, target, num_classes = 21, average = None)
iou2 = multiclass_jaccard_index(pred, target, num_classes = 21)

In [ ]:
print('mIU', mIU)
print('mIU1', mIU1)
print('iou', iou)
print('iou2', iou2)

In [ ]:
torch.mean(mIU, dim=0)

In [ ]:
# x = torch.zeros([21])
# print(x.shape)
# for i in range(21):
#     x[i] = torch.sum(torch.where(target == i, 1, 0))
#     print(x[i])
# print(torch.sum(x))

In [ ]:
4*224*224

In [ ]:
nc = 5
preds = torch.randint(0, 2, (4, nc, 16, 16))  # 4 samples, 5 classes, 16x16 prediction
target = torch.randint(0, 2, (4, nc, 16, 16))  # 4 samples, 5 classes, 16x16 target
print(mean_iou(preds, target, num_classes=nc))

print(mean_iou(preds, target, num_classes=nc, per_class=True))

In [ ]:
print(mean_iou(torch.argmax(preds, dim=1), torch.argmax(target, dim=1), num_classes=nc))

print(mean_iou(torch.argmax(preds, dim=1), torch.argmax(target, dim=1), num_classes=nc, per_class=True))

In [ ]:
multiclass_jaccard_index(preds, target, num_classes = 5, average = None)

In [ ]:
x = list(range(2,4))

In [ ]:
pred = torch.randint(5, (4,64,64))
target = torch.randint(5, (4,64,64))
target_oh = torch.permute(one_hot(target, num_classes = 5), (0,3,1,2))
pred_oh = torch.permute(one_hot(pred, num_classes = 5), (0,3,1,2))

In [ ]:
mIU = mean_iou(pred, target, num_classes=5, per_class=True)
mIU1 = mean_iou(pred_oh, target_oh, num_classes=5, per_class=True)
iou = multiclass_jaccard_index(pred, target, num_classes = 5, average = None)
iou2 = multiclass_jaccard_index(pred, target, num_classes = 5)

In [ ]:
# print('mIU', mIU)
print('Mean mIU', torch.mean(mIU, dim=0))
# print('mIU1', mIU1)
print('Mean mIU1', torch.mean(mIU1, dim=0))
print('iou', iou)
print('iou2', iou2)

In [ ]:
torch.mean(mIU1, dim=0) / iou

In [ ]:
z = (72 * torch.rand(21)) / torch.randint(1, 72, (21,))
z1 = (z <= 1.0)*1

In [ ]:
print(z)
print(z1)